In [8]:
!pip install opencv-contrib-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.1/56.1 MB 15.9 MB/s eta 0:00:0000:0100:01


# 안면인식

In [18]:
import cv2 #OpenCV 영상처리
import time

#classifier
path = './'
cascPath = './XML/haarcascade_frontalface_default.xml'
faceCascade = cv2.CascadeClassifier(cascPath)

#video caputure setting
capture = cv2.VideoCapture(0)
capture.set(cv2.CAP_PROP_FRAME_WIDTH,1280) #CAP_PROP_FRAME_WIDTH == 3
capture.set(cv2.CAP_PROP_FRAME_HEIGHT,960) #CAP_PROP_FRAME_HEIGHT == 4

#console message
face_id = input('\n enter user id end press <return> ==> ')
print("\n [INFO] Initializing face capture. Look the camera and wait ...")

count = 0 # # of caputre face images
#영상 처리 및 출력
while True: 
    ret, frame = capture.read() #카메라 상태 및 프레임
    # frame = cv2.flip(frame, 1) #좌우반전(1), 상하반전(0)
    
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(
        gray, #검출하고자 하는 원본이미지
        scaleFactor = 1.2, #검색 윈도우 확대 비율, 1보다 커야 한다
        minNeighbors = 6, #얼굴 사이 최소 간격(픽셀)
        minSize=(20,20) #얼굴 최소 크기. 이것보다 작으면 무시
    )

    #얼굴에 대해 rectangle 출력
    for (x,y,w,h) in faces:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
        #inputOutputArray, point1 , 2, colorBGR, thickness)
        count += 1
        cv2.imwrite("./dataset/object."+str(face_id)+'.'+str(count)+".jpg",
                    gray[y:y+h, x:x+w])

    #종료 조건
    if cv2.waitKey(1) > 0 : break #키 입력이 있을 때 반복문 종료
    if count >= 100 : break #100 face sample
    
print("\n [INFO] Exiting Program and cleanup stuff")

capture.release() #메모리 해제
cv2.destroyAllWindows()#모든 윈도우 창 닫기
cv2.waitKey(1)


 enter user id end press <return> ==> 1

 [INFO] Initializing face capture. Look the camera and wait ...

 [INFO] Exiting Program and cleanup stuff


-1

# 학습

In [19]:
import cv2
import numpy as np
from PIL import Image #python imaging library
import os

path = '/Users/werther/'
recognizer = cv2.face.LBPHFaceRecognizer_create()
detector = cv2.CascadeClassifier('./XML/haarcascade_frontalface_default.xml')

imagePaths = ['./dataset/'+ f for f in os.listdir('./dataset/') 
              if f.startswith('o')]

faceSamples = []
ids = []
for imagePath in imagePaths :
    src = cv2.imread(imagePath, 0)
    #user id
    id = int(os.path.split(imagePath)[-1].split(".")[1])
    
    #얼굴 샘플
    faces = detector.detectMultiScale(src)
    for(x,y,w,h) in faces:
        faceSamples.append(src[y:y+h,x:x+w])
        ids.append(id)
print(faceSamples)
print(ids)
print(len(faceSamples))
print(len(ids))

recognizer.train(faceSamples, np.array(ids))
recognizer.write('./trainer/trainer.yml')
print('\n [INFO] {0} faces trained. Exiting Program'.format(len(np.unique(ids))))

[array([[ 25,  32,  38, ...,  33,  36,  37],
       [ 40,  39,  27, ...,  29,  32,  33],
       [ 32,  35,  36, ...,  16,  24,  24],
       ...,
       [114, 131, 172, ..., 181, 211, 223],
       [ 89, 106, 137, ..., 204, 225, 229],
       [ 74,  86,  90, ..., 215, 219, 221]], dtype=uint8), array([[ 29,  29,  29, ...,  26,  37,  38],
       [ 29,  30,  29, ...,  24,  31,  27],
       [ 29,  29,  23, ...,  21,  27,  19],
       ...,
       [118, 110, 137, ..., 113, 101, 166],
       [ 95,  81,  91, ..., 100, 122, 184],
       [142, 131, 159, ..., 112, 160, 213]], dtype=uint8), array([[177, 177, 177, ...,  61,  61,  60],
       [177, 177, 177, ...,  59,  61,  61],
       [177, 177, 177, ...,  60,  61,  61],
       ...,
       [ 40,  38,  35, ...,  30,  29,  27],
       [ 39,  37,  35, ...,  30,  30,  30],
       [ 36,  36,  34, ...,  29,  31,  32]], dtype=uint8), array([[ 90,  90,  88, ..., 177, 173, 170],
       [ 89,  88,  88, ..., 188, 188, 188],
       [ 88,  88,  88, ..., 185, 186, 


 [INFO] 5 faces trained. Exiting Program


# 캠으로 얼굴 인식

In [47]:
import cv2
import numpy as np

recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read(path+'trainer/trainer.yml')
cascadePath = path+'xml/haarcascade_frontalface_default.xml'
faceCascade = cv2.CascadeClassifier(cascadePath)
font = cv2.FONT_HERSHEY_SIMPLEX

id = 0

names = ['None','SangRok Jung', 'SungWon Lee', 'YoungHoon Jung', 'UhJung Hwang', 
         'DaSol Choi', 'JaeHui Han', 'WonJoon Lee']

cam = cv2.VideoCapture(0)
cam.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cam.set(cv2.CAP_PROP_FRAME_HEIGHT, 960)

minW = 0.1 * cam.get(cv2.CAP_PROP_FRAME_WIDTH)
minH = 0.1 * cam.get(cv2.CAP_PROP_FRAME_HEIGHT)

while True:
    ret, img = cam.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    faces = faceCascade.detectMultiScale(
        gray,
        scaleFactor=1.2,
        minNeighbors=6,
        minSize=(int(minW), int(minH))
    )

    for(x,y,w,h) in faces:
        cv2.rectangle(img, (x,y), (x+w,y+h), (0,255,0),2)
        id, confidence = recognizer.predict(gray[y:y+h, x:x+w])

        if confidence < 55 :
            id = names[id]
        else:
            id = "unknown"
        
        confidence = "  {0}%".format(round(100-confidence))

        cv2.putText(img,str(id), (x+5,y-5),font,1,(255,255,255),2)
        cv2.putText(img,str(confidence), (x+5,y+h-5),font,1,(255,255,0),1)
    
    cv2.imshow('camera',img)
    if cv2.waitKey(1) > 0 : break

print("\n [INFO] Exiting Program and cleanup stuff")
cam.release()
cv2.destroyAllWindows()
cv2.waitKey(1)


 [INFO] Exiting Program and cleanup stuff


-1

In [22]:
!pip install pyautogui

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.0/59.0 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.8/227.8 kB 7.9 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.3/388.3 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 1.7 MB/s eta 0:00:00


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 732.3/732.3 kB 17.6 MB/s eta 0:00:0000:01
  Created wheel for pyautogui: filename=PyAutoGUI-0.9.53-py3-none-any.whl size=36601 sha256=8bed5a861b23f2a64f6483e5acb1bfa3ab69f1953a690227adf423ee6bd75d13
  Stored in directory: /Users/werther/Library/Caches/pip/wheels/ca/f8/81/e51ac3e5190f8e9d7c724f24006c3c25269ce8b6dcab354d12
  Created wheel for pygetwindow: filename=PyGetWindow-0.0.9-py3-none-any.whl size=11063 sha256=bbef5b5fb5aacbf3b72cdb7aff36e4edf051a1edf56558dcfe7b51ca91f3f6c2
  Stored in directory: /Users/werther/Library/Caches/pip/wheels/ab/ac/7c/49f3432609687efbee2433fefd53126870b8b541f8af44fd17
  Created wheel for pyscreeze: filename=PyScreeze-0.1.28-py3-none-any.whl size=13009 sha256=2d87060e12094fc44f2fb79f516bb57aff190858daef67755ff8c4c2cacd1950
  Stored in directory: /Users/werther/Library/Caches/pip/wheels/b2/cb/35/3998fb8c95724397e2967b697522ba660bbdf62b12d1da2d9a
  Created wheel for PyTweening: filename=pytweening-1.0.4-py3-none-a

  Attempting uninstall: pyobjc-core
    Found existing installation: pyobjc-core 8.5
    Uninstalling pyobjc-core-8.5:
      Successfully uninstalled pyobjc-core-8.5
  Attempting uninstall: pyobjc-framework-Cocoa
    Found existing installation: pyobjc-framework-Cocoa 8.5
    Uninstalling pyobjc-framework-Cocoa-8.5:
      Successfully uninstalled pyobjc-framework-Cocoa-8.5
  Attempting uninstall: pyobjc-framework-FSEvents
    Found existing installation: pyobjc-framework-FSEvents 8.5
    Uninstalling pyobjc-framework-FSEvents-8.5:
      Successfully uninstalled pyobjc-framework-FSEvents-8.5
  Attempting uninstall: pyobjc-framework-CoreServices
    Found existing installation: pyobjc-framework-CoreServices 8.5
    Uninstalling pyobjc-framework-CoreServices-8.5:
      Successfully uninstalled pyobjc-framework-CoreServices-8.5


In [ ]:

cv2.namedWindow(“result”);
cv2.moveWindow(“result”, 0, 500);

while 1:
    pic = pyautogui.screenshot(region=(0, 0, 1400, 1000))
    img_frame = np.array(pic)
    img_frame  = cv2.cvtColor(img_frame, cv2.COLOR_RGB2BGR)

cv.imshow(‘result’, img_frame)
    key = cv2.waitKey(1)
    if key == 27:
        break


In [ ]:
# pip install pyautogui
import cv2
import numpy as np
import pyautogui

recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read(path+'trainer/trainer.yml')
cascadePath = path+'xml/haarcascade_frontalface_default.xml'
faceCascade = cv2.CascadeClassifier(cascadePath)
font = cv2.FONT_HERSHEY_SIMPLEX

id = 0

names = ['None','SangRok Jung', 'SungWon Lee', 'YoungHoon Jung', 'UhJung Hwang', 
         'DaSol Choi', 'JaeHui Han', 'WonJoon Lee']

cam = cv2.VideoCapture(0)
cam.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cam.set(cv2.CAP_PROP_FRAME_HEIGHT, 960)

minW = 0.1 * cam.get(cv2.CAP_PROP_FRAME_WIDTH)
minH = 0.1 * cam.get(cv2.CAP_PROP_FRAME_HEIGHT)

while True:
    ret, img = cam.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    faces = faceCascade.detectMultiScale(
        gray,
        scaleFactor=1.2,
        minNeighbors=6,
        minSize=(int(minW), int(minH))
    )

    for(x,y,w,h) in faces:
        cv2.rectangle(img, (x,y), (x+w,y+h), (0,255,0),2)
        id, confidence = recognizer.predict(gray[y:y+h, x:x+w])

        if confidence < 55 :
            id = names[id]
        else:
            id = "unknown"
        
        confidence = "  {0}%".format(round(100-confidence))

        cv2.putText(img,str(id), (x+5,y-5),font,1,(255,255,255),2)
        cv2.putText(img,str(confidence), (x+5,y+h-5),font,1,(255,255,0),1)
    
    cv2.imshow('camera',img)
    if cv2.waitKey(1) > 0 : break

print("\n [INFO] Exiting Program and cleanup stuff")
cam.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

# 비디오에서 인물 검출
> 수정 필요

In [65]:
import cv2
import numpy as np


recognizer = cv2.face.LBPHFaceRecognizer_create()  
recognizer.read('./trainer/trainer.yml')
cascadePath = './XML/haarcascade_frontalface_default.xml'
faceCascade = cv2.CascadeClassifier(cascadePath)
font = cv2.FONT_HERSHEY_SIMPLEX

id = 0

names = ['None','Rok', 'Won', 'Sol', 'Jung', 'Jun', 'Hee', 'Hun']

cap = cv2.VideoCapture('./image/academy_front1.mp4')
frame_size = (int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)), 
              int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
# cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
# cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 960)
minW = 0.1 * cap.get(cv2.CAP_PROP_FRAME_WIDTH)
minH = 0.1 * cap.get(cv2.CAP_PROP_FRAME_HEIGHT)

while True: 
    retval, frame = cap.read() #카메라 상태 및 프레임
#     frame = cv2.flip(frame, -1) # 상하반전
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    faces = faceCascade.detectMultiScale(
        gray,
        scaleFactor=1.2,
        minNeighbors=7,
        minSize=(int(minW), int(minH))
    )

    for(x,y,w,h) in faces:
        cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,0),2)
        id, confidence = recognizer.predict(gray[y:y+h, x:x+w])

        if confidence < 40 :
            id = names[id]
        else:
            id = "unknown"
        
        confidence = "  {0}%".format(round(100-confidence))

        cv2.putText(frame,str(id), (x+5,y-5),font,1,(255,255,255),2)
        cv2.putText(frame,str(confidence), (x+5,y+h-5),font,1,(255,255,0),1)
    
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) > 0 : break

print("\n [INFO] Exiting Program and cleanup stuff")

cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

error: OpenCV(4.6.0) /Users/runner/work/opencv-python/opencv-python/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


# QR Code 검출

In [67]:
import webbrowser
import numpy

inputImage = cv2.imread('./image/QR_sangrok.png')
qrDecoder = cv2.QRCodeDetector()

# QR코드를 찾고 디코드해줍니다.
data, bbox, _ = qrDecoder.detectAndDecode(inputImage) # qr code 찾고 번역

if len(data) > 0:
    print('Decoded Data: {}'.format(data))
    
    if bbox is not None:
        nrOfPoints = len(bbox)
    cv2.line(inputImage, tuple(bbox[0][0].astype(np.int)), tuple(bbox[0][1].astype(np.int)), (0,0,255), 3)
    cv2.line(inputImage, tuple(bbox[0][1].astype(np.int)), tuple(bbox[0][2].astype(np.int)), (0,0,255), 3)
    cv2.line(inputImage, tuple(bbox[0][2].astype(np.int)), tuple(bbox[0][3].astype(np.int)), (0,0,255), 3)
    cv2.line(inputImage, tuple(bbox[0][3].astype(np.int)), tuple(bbox[0][0].astype(np.int)), (0,0,255), 3)
    
    # display
    cv2.imshow('QR Code', inputImage)
    cv2.waitKey()
    cv2.destroyAllWindows()
    cv2.waitKey(1)
    
    # web browser
    webbrowser.open(data) # data가 사이트면 웹브라우저를 이용해서 띄운다
else :
    print('QR Code not detected')

Decoded Data: https://qrco.de/bdWMTy


/var/folders/t4/vwr916y50fv3wcm0xlgfkyg00000gn/T/ipykernel_1815/2068405865.py:15: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  cv2.line(inputImage, tuple(bbox[0][0].astype(np.int)), tuple(bbox[0][1].astype(np.int)), (0,0,255), 3)
/var/folders/t4/vwr916y50fv3wcm0xlgfkyg00000gn/T/ipykernel_1815/2068405865.py:16: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `n

# 영상분할

## 치아

In [93]:
from matplotlib import pyplot as plt

src = cv2.imread('./image/mandible.png')
src_hsv = cv2.cvtColor(src, cv2.COLOR_BGR2HSV)

dst = cv2.inRange(src, (0, 128, 0), (100, 255, 100))
dst2 = cv2.inRange(src_hsv, (-10, 40, 40), (30,255,255))

cv2.imshow('src', src)
cv2.imshow('src_hsv', src_hsv)
cv2.imshow('dst2', dst2)
cv2.waitKey()
cv2.destroyAllWindows()
cv2.waitKey(1)
    


-1

# brain MRI

In [131]:
import cv2
import numpy as np 
from matplotlib import pyplot as plt

src = cv2.imread('./image/brain2.png', 0)

blur = cv2.GaussianBlur(src,(9,9),0)

ret, th = cv2.threshold(src, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
ret2, th2 = cv2.threshold(blur, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
ret2, th3 = cv2.threshold(blur, 200, 255, cv2.THRESH_OTSU)

# src_hsv = cv2.cvtColor(th2, cv2.COLOR_BGR2HSV)
# dst = cv2.inRange(th2, (0, 128, 0), (100, 255, 100))
# dst2 = cv2.inRange(src_hsv, (-10, 100, 100), (30,255,255))

cv2.imshow('src', src)
cv2.imshow('th', th)
cv2.imshow('th2', th2)
cv2.imshow('th3', th3)
cv2.waitKey()
cv2.destroyAllWindows()
cv2.waitKey(1)

-1